In [1]:
import numpy as np
import pandas as pd
from Classes.PercepTeacher import PerceptronTeacher
from Functions.function_of_activation import sigmoid, dsigmoid
from Functions.errorFunctions import MSE
from Classes.Layer import Layer
from Classes.LayerGen import LayerGenerator
from random import random
from Classes.Perceptron import Perceptron
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
np.random.seed(1)


In [2]:
models = pd.read_csv('../Data/DataLaba2/neo_task_smoted.csv')
X_train, X_test, y_train, y_test = train_test_split(models.drop(
    'hazardous', axis=1), models['hazardous'], test_size=0.1, random_state=42)


In [3]:
def act(w, x, d):
    print('1: ', w.dot(x.T))
    print(d)
    return w.dot(x.T) + d.T


In [4]:
layer_mass = LayerGenerator(sizes_of_input_layer=X_train.shape[1],
                            sizes_of_hidden_layers=[X_train.shape[1] // 2, X_train.shape[1] // 4],
                            sizes_of_output_layer=1,
                            summators=[act, 
                                       act,
                                       act,
                                       act],
                            dsummators=[lambda w, x, d: (x, d),
                                        lambda w, x, d: (x, d),
                                        lambda w, x, d: (x, d),
                                        lambda w, x, d: (x, d)],
                            activation_functions=[sigmoid,
                                                  sigmoid,
                                                  sigmoid,
                                                  sigmoid],
                            dactivation_functions=[dsigmoid,
                                                   dsigmoid,
                                                   dsigmoid,
                                                   dsigmoid],
                            function_of_generation=lambda a, b: np.random.rand(a, b)).generate()
layer_mass


In [5]:
for l in layer_mass:
    print('w: ', l.get_weights(), 'd: ', l.get_dias())


w:  [[4.17022005e-01 7.20324493e-01 1.14374817e-04 3.02332573e-01]
 [1.46755891e-01 9.23385948e-02 1.86260211e-01 3.45560727e-01]] d:  [[0.39676747 0.53881673]]
w:  [[0.41919451 0.6852195 ]] d:  [[0.20445225]]
w:  [[0.87811744]] d:  [[0.02738759]]


In [6]:
X_train.shape


(147592, 4)

In [7]:
def batch_spliter(x, y, size, num):
    batches = list()
    for _ in range(num):
        js = np.random.shuffle(np.arange(size))
        batches.append((x[js][0], y[js][0]))
    return batches


# Not trained

In [8]:
k = []
for i in layer_mass:
    k.append(i.get_weights())
    print(i.get_weights())


[[4.17022005e-01 7.20324493e-01 1.14374817e-04 3.02332573e-01]
 [1.46755891e-01 9.23385948e-02 1.86260211e-01 3.45560727e-01]]
[[0.41919451 0.6852195 ]]
[[0.87811744]]


In [9]:
a = Perceptron(layer_mass)
res = [a.predict(i)[0] for i in X_train.to_numpy()[:200]]
print(MSE(res, y_train.to_numpy()[:200]))
for p, y in zip(res, y_train.to_numpy()[:200]):
    print(f'p:{p},\t y:{y},\t d:{abs(p-y)}')


1:  [[-0.67899219]
 [-0.03524061]]
[[0.39676747 0.53881673]]
1:  [[0.60731214]]
[[0.20445225]]
1:  [[0.60808347]]
[[0.02738759]]
1:  [[-0.93220291]
 [-0.21938044]]
[[0.39676747 0.53881673]]
1:  [[0.55165773]]
[[0.20445225]]
1:  [[0.59756644]]
[[0.02738759]]
1:  [[-0.99087142]
 [-0.3210512 ]]
[[0.39676747 0.53881673]]
1:  [[0.5288722]]
[[0.20445225]]
1:  [[0.59319851]]
[[0.02738759]]
1:  [[-0.70157365]
 [-0.05979849]]
[[0.39676747 0.53881673]]
1:  [[0.60103304]]
[[0.20445225]]
1:  [[0.60690789]]
[[0.02738759]]
1:  [[-0.95267826]
 [-0.17747238]]
[[0.39676747 0.53881673]]
1:  [[0.5566389]]
[[0.20445225]]
1:  [[0.59851658]]
[[0.02738759]]
1:  [[-0.98588692]
 [-0.25866464]]
[[0.39676747 0.53881673]]
1:  [[0.53987381]]
[[0.20445225]]
1:  [[0.59531192]]
[[0.02738759]]
1:  [[-0.65841944]
 [-0.04917984]]
[[0.39676747 0.53881673]]
1:  [[0.60718227]]
[[0.20445225]]
1:  [[0.60805918]]
[[0.02738759]]
1:  [[-0.79072089]
 [-0.13094643]]
[[0.39676747 0.53881673]]
1:  [[0.58036443]]
[[0.20445225]]
1:  

In [10]:
np.array(res).mean(), np.array(res).std(), np.array(res).min()


(0.6523271999671126, 0.0012989250104181813, 0.649346628017081)

In [11]:
# -1.1027074438226645
r2_score(np.array(res) >= np.array(res).mean(), y_train.to_numpy()[:200])


-1.3621259133219898

# Trained

In [12]:
teacher = PerceptronTeacher(layers=layer_mass,
                                 step=0.1,
                                 batch_spliter=lambda a, b: batch_spliter(a, b, a.shape[0]//2, 1),
                                 epochs=15,
                                 error_function=MSE,
                                 derror_function=lambda x, y: 2 * x - 2 * y)


In [13]:
tl = teacher.teach(X_train.to_numpy()[:200], y_train.to_numpy()[:200])


1:  [[-0.67899219]
 [-0.03524061]]
[[0.39676747 0.53881673]]
1:  [[0.60731214]]
[[0.20445225]]
1:  [[0.60808347]]
[[0.02738759]]
df:  (1, 1) [[0.22636740339557113]]
d_output:  (1, 1) [[-0.6925420574814898]]
a:  (1, 1) [[0.6924853574786082]]
s:  (1, 1) [[0.635471060045309]]
W:  (1, 1) [[0.8781174363909454]]
B:  (1, 1) [[0.027387593197926163]]

res
1)  (1, 1) [[-0.1567689472943112]]
2.1)  (1, 1) [[0.6924853574786082]]
2.2)  (1, 1) [[0.027387593197926163]]
W_Step:  (1, 1) [[-0.010856020050864619]]
df:  (1, 1) [[0.2129493871563324]]
d_output:  (1, 1) [[-0.10856020050864618]]
a:  (1, 2) [[0.4299084414639844, 0.6232993646531]]
s:  (1, 1) [[0.8117643893341044]]
W:  (1, 2) [[0.4191945144032948, 0.6852195003967595]]
B:  (1, 1) [[0.20445224973151743]]

res
1)  (1, 1) [[-0.02311782816788477]]
2.1)  (1, 2) [[0.4299084414639844, 0.6232993646531]]
2.2)  (1, 1) [[0.20445224973151743]]
W_Step:  (1, 2) [[-0.0004969274738843769, -0.0007204663804601058]]
df:  (2, 1) [[0.24508717342199227], [0.23479726667

ValueError: cannot reshape array of size 4 into shape (1,2)

In [ ]:
for i in tl:
    print(i.get_weights())


[[0.42195889 0.74705639 0.01396987 0.25014061]
 [0.15169277 0.11907049 0.2001157  0.29336876]]
[[0.41602208 0.60889017]]
[[0.33143238]]


In [ ]:
for i, j in zip(k, tl):
    print(i - j.get_weights())


[[-0.00493688 -0.02673189 -0.01385549  0.05219197]
 [-0.00493688 -0.02673189 -0.01385549  0.05219197]]
[[0.00317244 0.07632933]]
[[0.54668506]]


In [ ]:
a = Perceptron(tl)
res = [a.predict(i)[0] for i in X_train.to_numpy()[:200]]
print(MSE(res, y_train.to_numpy()[:200]))
for p, y in zip(res, y_train.to_numpy()[:200]):
    print(f'p:{p},\t y:{y},\t d:{abs(p-y)}')


48.94284019699347
p:[0.5572098],	 y:1.0,	 d:[0.4427902]
p:[0.55596741],	 y:0.0,	 d:[0.55596741]
p:[0.55572871],	 y:1.0,	 d:[0.44427129]
p:[0.55717584],	 y:1.0,	 d:[0.44282416]
p:[0.55629539],	 y:1.0,	 d:[0.44370461]
p:[0.55582711],	 y:0.0,	 d:[0.55582711]
p:[0.55725951],	 y:1.0,	 d:[0.44274049]
p:[0.55654976],	 y:0.0,	 d:[0.55654976]
p:[0.55783101],	 y:1.0,	 d:[0.44216899]
p:[0.55555311],	 y:1.0,	 d:[0.44444689]
p:[0.55697801],	 y:1.0,	 d:[0.44302199]
p:[0.55691604],	 y:1.0,	 d:[0.44308396]
p:[0.55586828],	 y:1.0,	 d:[0.44413172]
p:[0.55587748],	 y:1.0,	 d:[0.44412252]
p:[0.55649114],	 y:0.0,	 d:[0.55649114]
p:[0.55738975],	 y:0.0,	 d:[0.55738975]
p:[0.55702577],	 y:1.0,	 d:[0.44297423]
p:[0.55564003],	 y:0.0,	 d:[0.55564003]
p:[0.55710676],	 y:1.0,	 d:[0.44289324]
p:[0.55716472],	 y:1.0,	 d:[0.44283528]
p:[0.55641842],	 y:1.0,	 d:[0.44358158]
p:[0.55654292],	 y:0.0,	 d:[0.55654292]
p:[0.55690287],	 y:1.0,	 d:[0.44309713]
p:[0.55577128],	 y:0.0,	 d:[0.55577128]
p:[0.55718555],	 y:1.0,	

In [ ]:
np.array(res).mean(), np.array(res).std(), np.array(res).min()


(0.5565905829037355, 0.0005425376882252553, 0.5554459803744799)

In [ ]:
# -1.1027074438226645
r2_score(np.array(res) >= np.mean(res)-1e-4, y_train.to_numpy()[:200])


-1.1135265700483097